In [ ]:
# Patch Embedding & Positional Embedding

class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        assert img_size % patch_size == 0, # 이미지 사이즈가 나누어 떨어지는지 한번더 체크
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = img_size // patch_size  
        self.num_patches = self.grid_size * self.grid_size 

        # Conv2d를 활용하면 linear projection 과정까지 함께 할 수 있음
        self.proj = nn.Conv2d(in_chans, embed_dim,
                              kernel_size=patch_size, stride=patch_size)

        # cls토큰이랑 위치 벡터 정의
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, embed_dim)) 
        #self.num_patches + 1인 이유: cls 토큰 더 해줘서

        
        # cls 토큰이랑 위치 벡터 정규화로 초기와 해줌
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)

    def forward(self, x):

        
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)

        # CLS 토큰 붙이기
        B = x.shape[0]
        cls = self.cls_token.expand(B, -1, -1) 
        x = torch.cat([cls, x], dim=1)            

        # 위치 임베딩 더하기
        x = x + self.pos_embed        
        return x  # tokens